# Check Workspace and Import Packages

In [ ]:
import os
import subprocess
import numpy as np
import pandas as pd
import pandas
from datetime import datetime
now = datetime.now()

In [ ]:
# Get the bucket name
my_bucket = os.getenv('WORKSPACE_BUCKET')

# List objects in the bucket
print(subprocess.check_output(f"gsutil ls -r {my_bucket}", shell=True).decode('utf-8'))

----

Create function to set seeds for reproducibility

----

In [ ]:
def set_seeds(offset=0):
    import os
    import random
    import numpy as np
    import tensorflow as tf

    # Set the seed for numpy
    np.random.seed(42+offset)

    # Set the seed for the Python random module
    random.seed(42+offset)

    # Set the seed for TensorFlow
    tf.random.set_seed(42+offset)

    # Ensure reproducibility with certain environment variables
    os.environ['PYTHONHASHSEED'] = str(42+offset)


    ### Hold off on more extensive seeds (below) until verified necessary


    # # Configure TensorFlow to use a single thread if required
    # tf.config.threading.set_intra_op_parallelism_threads(1)
    # tf.config.threading.set_inter_op_parallelism_threads(1)

    # # Optionally, set environment variables to control NumPy threading behavior
    # os.environ['OMP_NUM_THREADS'] = '1'
    # os.environ['MKL_NUM_THREADS'] = '1'

    # # Example to demonstrate reproducibility
    # print("Numpy Random:", np.random.rand(3))
    # print("Python Random:", random.random())

    # # TensorFlow example
    # tf_example = tf.random.uniform([3])
    # print("TensorFlow Random:", tf_example)

    # # PyTorch Example (if using PyTorch)
    # import torch

    # torch.manual_seed(42+offset)
    # if torch.cuda.is_available():
    #     torch.cuda.manual_seed(42+offset)
    #     torch.cuda.manual_seed_all(42+offset)  # if using multi-GPU.
    #     torch.backends.cudnn.deterministic = True  # cuDNN
    #     torch.backends.cudnn.benchmark = False

    # # Generate reproducible random numbers with PyTorch
    # print("PyTorch Random:", torch.rand(3))

----

Create function to start/stop logging RAM usage to file

----

In [ ]:
import os
import psutil
import threading
import time
from google.cloud import storage

def log_memory_usage(stop_event, file_name):
    with open(file_name, 'w') as f:
        while not stop_event.is_set():
            # Log memory usage to a local file
            memory_info = psutil.virtual_memory()
            gb_used = memory_info.used / (1024 ** 3)
            mem_usage = f"{time.ctime()}: {gb_used:.2f} GB\n"
            print(mem_usage)
            f.write(mem_usage)
            f.flush()
            
            # Upload the local file to GCS
            try:
                destination_blob_name = f'logs/{file_name}'
                upload_to_gcs(file_name, destination_blob_name)
            except Exception as e:
                print(f"Failed to upload to GCS: {e}")
                
            time.sleep(30)
            
def upload_to_gcs(source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    # Get the bucket name
    my_bucket = os.getenv('WORKSPACE_BUCKET')
    # Initialize a storage client
    storage_client = storage.Client()
    bucket = storage_client.bucket(my_bucket[5:])
    blob = bucket.blob(destination_blob_name)

    # Upload the file
    blob.upload_from_filename(source_file_name)

#     print(f"File {source_file_name} uploaded to {destination_blob_name}.")

In [ ]:
stop_event = threading.Event()
memory_thread = None
thread_lock = threading.Lock()  # To ensure thread-safe operations

def RAM_start():
    global stop_event
    global memory_thread

    with thread_lock:
        # Clear the stop event if it is set
        if stop_event.is_set():
            stop_event.clear()

        file_name = 'memory_usage.txt'
        
        # Stop the existing thread if it is running
        if memory_thread and memory_thread.is_alive():
            RAM_stop()
        
        # Create and start a new memory logging thread
        memory_thread = threading.Thread(target=log_memory_usage, args=(stop_event, file_name))
        memory_thread.start()
        print("Memory logging started")

def RAM_stop():
    global stop_event
    global memory_thread

    with thread_lock:
        # Set the stop event to signal the thread to stop
        stop_event.set()

        # Wait for the thread to finish if it exists
        if memory_thread:
            memory_thread.join()
            memory_thread = None  # Reset the thread to None
            print("Memory logging stopped")

In [ ]:
RAM_start()

---

---

---

# Experiment

## Experiment Functions

In [ ]:
## 1. Hocevar T, Zupan B, Stålring J. Conformal Prediction with Orange. Journal of Statistical Software. 2021;98(7). doi:https://doi.org/10.18637/jss.v098.i07
##  => https://github.com/biolab/orange3-conformal

# nonconformity [1]
def inverse_probability(probs, target_idx):
    return 1 - probs[target_idx]
  
# nonconformity [1]
def probability_margin(probs, target_idx):
    # prediciton probabilities for one example at a time
    assert probs.ndim == 1
    
    py = probs[target_idx]
    pz = max(prob for label_idx, prob in enumerate(probs) if label_idx != target_idx)
    return (1.0 - (py - pz)) / 2

# classification [1]
class PredictionClass:
    """Conformal classification prediction object,
    which is produced by the :py:func:`ConformalClassifier.predict` method.

    Attributes:
        p (List): List of pairs (p-value, class)
        eps (float): Default significance level (error rate).

    Examples:
        >>> train, test = next(LOOSampler(Table('iris')))
        >>> tcp = TransductiveClassifier(InverseProbability(NaiveBayesLearner()), train)

        >>> prediction = tcp.predict(test[0], 0.1)
        >>> print(prediction.confidence(), prediction.credibility())

        >>> prediction = tcp.predict(test[0])
        >>> print(prediction.classes(0.1), prediction.classes(0.9))
    """

    def __init__(self, p, eps):
        """Initialize the prediction.

        Args:
            p (List): List of pairs (p-value, class)
            eps (float): Default significance level (error rate).
        """
        self.p = p
        self.eps = eps

    def classes(self, eps=None):
        """ Compute the set of classes under the default or given `eps` value.

        Args:
            eps (float): Significance level (error rate).

        Returns:
            List of predicted classes.
        """
        if eps is None:
            eps = self.eps
            assert(eps is not None)
        return [y for p_y, y in self.p if p_y > eps]

    def verdict(self, ref, eps=None):
        """Conformal classification prediction is correct when the actual class appears
        among the predicted classes.

        Args:
            ref: Reference/actual class
            eps (float): Significance level (error rate).

        Returns:
            True if the prediction with default or specified `eps` is correct.
        """
        return ref in self.classes(eps)

    def confidence(self):
        """Confidence is an efficiency measure of a single prediction.

        Computes minimum :math:`\\mathit{eps}` that would still result in a prediction of a single label.
        :math:`\\mathit{eps} = \\text{second\_largest}(p_i)`

        Returns:
            float: Confidence :math:`1-\\mathit{eps}`.
        """
        return 1-sorted([p_y for p_y, y in self.p], reverse=True)[1]

    def credibility(self):
        """Credibility is an efficiency measure of a single prediction.
        Small credibility indicates an unusual example.

        Computes minimum :math:`\\mathit{eps}` that would result in an empty prediction set.
        :math:`\\mathit{eps} = \\text{max}(p_i)`

        Returns:
            float: Credibility :math:`\\mathit{eps}`.
        """
        return max(p_y for p_y, y in self.p)

####

def set_seeds(offset=0):
    import os
    import random
    import numpy as np
    import tensorflow as tf

    # Set the seed for numpy
    np.random.seed(42+offset)

    # Set the seed for the Python random module
    random.seed(42+offset)

    # Set the seed for TensorFlow
    tf.random.set_seed(42+offset)

    # Ensure reproducibility with certain environment variables
    os.environ['PYTHONHASHSEED'] = str(42+offset)
    
    ### Hold off on more extensive seeds (below) until verified necessary


    # # Configure TensorFlow to use a single thread if required
    # tf.config.threading.set_intra_op_parallelism_threads(1)
    # tf.config.threading.set_inter_op_parallelism_threads(1)

    # # Optionally, set environment variables to control NumPy threading behavior
    # os.environ['OMP_NUM_THREADS'] = '1'
    # os.environ['MKL_NUM_THREADS'] = '1'

    # # Example to demonstrate reproducibility
    # print("Numpy Random:", np.random.rand(3))
    # print("Python Random:", random.random())

    # # TensorFlow example
    # tf_example = tf.random.uniform([3])
    # print("TensorFlow Random:", tf_example)

    # # PyTorch Example (if using PyTorch)
    # import torch

    # torch.manual_seed(42+offset)
    # if torch.cuda.is_available():
    #     torch.cuda.manual_seed(42+offset)
    #     torch.cuda.manual_seed_all(42+offset)  # if using multi-GPU.
    #     torch.backends.cudnn.deterministic = True  # cuDNN
    #     torch.backends.cudnn.benchmark = False

    # # Generate reproducible random numbers with PyTorch
    # print("PyTorch Random:", torch.rand(3))
    print()
    
# Function to get a random chunk of 10 contiguous days with no NaNs in specified columns
def get_random_chunk(df, chunk_size=10, max_attempts=10):
    columns_to_check = ['std_hr', 'morning_hr', 'afternoon_hr', 'evening_hr', 'night_hr']
    df = df.sort_values(by='date').reset_index(drop=True)
    if len(df) < chunk_size:
        return None  # Not enough data to get the desired chunk size
    
    attempts = 0
    while attempts < max_attempts:
        start_idx = np.random.randint(0, len(df) - chunk_size + 1)
        chunk = df.iloc[start_idx:start_idx + chunk_size]
        if not chunk[columns_to_check].isna().any().any():
            return chunk
        attempts += 1
    
    return None  # No valid chunk found after max_attempts

def get_random_contiguous_chunks_with_no_missing_data(df, chunk_size=10, max_attempts=10):
    # Apply the function to each person
    grouped = df.groupby('person_id')
    chunks = grouped.apply(lambda x: get_random_chunk(x, chunk_size, max_attempts)).dropna()

    # Drop the multi-level index created by groupby and apply
    chunks.reset_index(drop=True, inplace=True)
    
    return chunks

def one_hot_encode_categorical_columns(df, columns_to_encode):
    from sklearn.preprocessing import OneHotEncoder

    # Initialize OneHotEncoder
    encoder = OneHotEncoder(sparse_output=False, drop=None)

    # Fit and transform the selected columns
    encoded_data = encoder.fit_transform(df[columns_to_encode])

    ohe_cols = encoder.get_feature_names_out(columns_to_encode)
    
    # Create a DataFrame with the encoded columns
    encoded_df = pd.DataFrame(encoded_data, columns=ohe_cols)

    # Reset the index to avoid misalignment when concatenating
    encoded_df.index = df.index
    
    # Drop the original columns from the DataFrame
    df = df.drop(columns=columns_to_encode)
    
    # Concatenate the original DataFrame with the encoded DataFrame
    df = pd.concat([df, encoded_df], axis=1)
    
    ohe_col_list = ohe_cols.tolist()
    
    return df, ohe_col_list

def merge_demographics_and_ohe_categorical_columns(df, demographic_df):
    df = df.merge(demographic_df, on='person_id')
    cols = [col for col in demographic_df.columns 
            if col not in ['person_id','age']]     # ['gender', 'race', 'ethnicity', 'sex_at_birth']
    
    return one_hot_encode_categorical_columns(df, cols)

def standard_scale_except_excluded(train, cal, test, exclude_list):
    from sklearn.preprocessing import StandardScaler
    
    # Columns to scale
    columns_to_scale = [col for col in train.columns if col not in exclude_list]
    
    # Initialize the scaler
    scaler = StandardScaler()

    # Scale the selected columns
    train.loc[:, columns_to_scale] = scaler.fit_transform(train.loc[:, columns_to_scale])

    cal.loc[:, columns_to_scale] = scaler.transform(cal.loc[:, columns_to_scale])
    test.loc[:, columns_to_scale] = scaler.transform(test.loc[:, columns_to_scale])

    return train, cal, test

def get_scaled_reshaped_train_calibrate_test(df_X, df_y, num_days, random_offset, exclude_scale_cols):
    from sklearn.model_selection import train_test_split
    
    # Get unique person_ids
    person_ids = df_X['person_id'].unique()

    # Split into train, calibration, and test
    person_ids_train, person_ids_temp = train_test_split(person_ids, test_size=0.6, random_state=42+random_offset)
    person_ids_calibration, person_ids_test = train_test_split(person_ids_temp, test_size=0.5, random_state=42+random_offset)

    # Filter the original dataframe to create the splits
    df_X_train = df_X[df_X['person_id'].isin(person_ids_train)]
    df_X_calibration = df_X[df_X['person_id'].isin(person_ids_calibration)]
    df_X_test = df_X[df_X['person_id'].isin(person_ids_test)]

    # Similarly, filter the labels
    df_y_train = df_y.loc[person_ids_train]
    df_y_calibration = df_y.loc[person_ids_calibration]
    df_y_test = df_y.loc[person_ids_test]
    
    # Standard scale (only fitting train)
    df_X_train, df_X_calibration, df_X_test = standard_scale_except_excluded(df_X_train, 
                                                                             df_X_calibration, 
                                                                             df_X_test, 
                                                                             exclude_list=[
                                                                                 'person_id',
                                                                                 'date',
                                                                                 *exclude_scale_cols])
    
    # Drop the person_id from the features (assuming it's the first column)
    X_train = df_X_train.drop(columns=['person_id','date']).values.reshape(len(person_ids_train), num_days, -1)
    X_calibration = df_X_calibration.drop(columns=['person_id','date']).values.reshape(len(person_ids_calibration), num_days, -1)
    X_test = df_X_test.drop(columns=['person_id','date']).values.reshape(len(person_ids_test), num_days, -1)

    # Ensure that labels are aligned with the X splits
    y_train = df_y_train.loc[person_ids_train].values
    y_calibration = df_y_calibration.loc[person_ids_calibration].values
    y_test = df_y_test.loc[person_ids_test].values

    return (X_train, y_train, person_ids_train), (X_calibration, y_calibration, person_ids_calibration), (X_test, y_test, person_ids_test)
 
def does_this_experiments_input_data_exist(run_num, num_days, with_demographics):
    from google.cloud import storage
    import os
    import numpy as np
    import shutil
             
    client = storage.Client()
    bucket = client.get_bucket(os.getenv('WORKSPACE_BUCKET')[5:])
    
    base_path = f'data/experiments/{run_num}'
    suffix = f'_{str(num_days)}_days_{"with" if with_demographics else "wout"}_demo.npy'

    # Helper function to check if a file exists in GCS
    def gcs_file_exists(bucket, file_path):
        return storage.Blob(bucket=bucket, name=file_path).exists(client)

    # Check if all necessary files exist in GCS
    files_exist = all([
        gcs_file_exists(bucket, f'{base_path}/X_train{suffix}'),
        gcs_file_exists(bucket, f'{base_path}/X_cal{suffix}'),
        gcs_file_exists(bucket, f'{base_path}/X_test{suffix}'),
        gcs_file_exists(bucket, f'{base_path}/y_train{suffix}'),
        gcs_file_exists(bucket, f'{base_path}/y_cal{suffix}'),
        gcs_file_exists(bucket, f'{base_path}/y_test{suffix}'),
        gcs_file_exists(bucket, f'{base_path}/ids_train{suffix}'),
        gcs_file_exists(bucket, f'{base_path}/ids_cal{suffix}'),
        gcs_file_exists(bucket, f'{base_path}/ids_test{suffix}')
    ])

    if files_exist:
        local_base_path = f'data/experiments/{run_num}'

        # Check if all necessary files exist locally
        local_files_exist = all([
            os.path.exists(f'{local_base_path}/X_train{suffix}'),
            os.path.exists(f'{local_base_path}/X_cal{suffix}'),
            os.path.exists(f'{local_base_path}/X_test{suffix}'),
            os.path.exists(f'{local_base_path}/y_train{suffix}'),
            os.path.exists(f'{local_base_path}/y_cal{suffix}'),
            os.path.exists(f'{local_base_path}/y_test{suffix}'),
            os.path.exists(f'{local_base_path}/ids_train{suffix}'),
            os.path.exists(f'{local_base_path}/ids_cal{suffix}'),
            os.path.exists(f'{local_base_path}/ids_test{suffix}')
        ])

        if not local_files_exist:
            # Download missing files from GCS to local directory
            def download_from_gcs(bucket, remote_file, local_file):
                blob = bucket.blob(remote_file)
                blob.download_to_filename(local_file)
                print(f'Downloaded {remote_file} from GCS to {local_file}.')

            download_from_gcs(bucket, f'{base_path}/X_train{suffix}', f'{local_base_path}/X_train{suffix}')
            download_from_gcs(bucket, f'{base_path}/X_cal{suffix}', f'{local_base_path}/X_cal{suffix}')
            download_from_gcs(bucket, f'{base_path}/X_test{suffix}', f'{local_base_path}/X_test{suffix}')
            download_from_gcs(bucket, f'{base_path}/y_train{suffix}', f'{local_base_path}/y_train{suffix}')
            download_from_gcs(bucket, f'{base_path}/y_cal{suffix}', f'{local_base_path}/y_cal{suffix}')
            download_from_gcs(bucket, f'{base_path}/y_test{suffix}', f'{local_base_path}/y_test{suffix}')
            download_from_gcs(bucket, f'{base_path}/ids_train{suffix}', f'{local_base_path}/ids_train{suffix}')
            download_from_gcs(bucket, f'{base_path}/ids_cal{suffix}', f'{local_base_path}/ids_cal{suffix}')
            download_from_gcs(bucket, f'{base_path}/ids_test{suffix}', f'{local_base_path}/ids_test{suffix}')

        # Load the numpy arrays from the local files
        X_train = np.load(f'{local_base_path}/X_train{suffix}')
        X_cal = np.load(f'{local_base_path}/X_cal{suffix}')
        X_test = np.load(f'{local_base_path}/X_test{suffix}')
        y_train = np.load(f'{local_base_path}/y_train{suffix}')
        y_cal = np.load(f'{local_base_path}/y_cal{suffix}')
        y_test = np.load(f'{local_base_path}/y_test{suffix}')
        ids_train = np.load(f'{local_base_path}/ids_train{suffix}')
        ids_cal = np.load(f'{local_base_path}/ids_cal{suffix}')
        ids_test = np.load(f'{local_base_path}/ids_test{suffix}')

        return True, X_train, X_cal, X_test, y_train, y_cal, y_test, ids_train, ids_cal, ids_test
    else:
        return False, None, None, None, None, None, None, None, None, None
           
def write_this_experiments_input_data_if_not_exists(run_num, num_days, with_demographics, 
                                                    X_train, y_train, ids_train, 
                                                    X_cal, y_cal, ids_cal, 
                                                    X_test, y_test, ids_test):
    from google.cloud import storage
    import numpy as np
    import os
             
    base_path = f'data/experiments/{run_num}'
    suffix = f'_{str(num_days)}_days_{"with" if with_demographics else "wout"}_demo.npy'
    
    def save_array_if_not_exists(array, filename):
        directory = os.path.dirname(filename)
        if not os.path.exists(directory):
            os.makedirs(directory)
        np.save(filename, array)
    
    client = storage.Client()
    bucket = client.get_bucket(os.getenv('WORKSPACE_BUCKET')[5:]) 

    # Helper function to check if a file exists in GCS
    def gcs_file_exists(bucket, file_path):
        return storage.Blob(bucket=bucket, name=file_path).exists(client)
    
    # Check if all necessary files exist in GCS
    files_exist = all([
        gcs_file_exists(bucket, f'{base_path}/X_train{suffix}'),
        gcs_file_exists(bucket, f'{base_path}/X_cal{suffix}'),
        gcs_file_exists(bucket, f'{base_path}/X_test{suffix}'),
        gcs_file_exists(bucket, f'{base_path}/y_train{suffix}'),
        gcs_file_exists(bucket, f'{base_path}/y_cal{suffix}'),
        gcs_file_exists(bucket, f'{base_path}/y_test{suffix}'),
        gcs_file_exists(bucket, f'{base_path}/ids_train{suffix}'),
        gcs_file_exists(bucket, f'{base_path}/ids_cal{suffix}'),
        gcs_file_exists(bucket, f'{base_path}/ids_test{suffix}')
    ])

    if not files_exist:
        # Save the files locally first
        save_array_if_not_exists(X_train, f'{base_path}/X_train{suffix}')
        save_array_if_not_exists(y_train, f'{base_path}/y_train{suffix}')
        save_array_if_not_exists(ids_train, f'{base_path}/ids_train{suffix}')
        save_array_if_not_exists(X_cal, f'{base_path}/X_cal{suffix}')
        save_array_if_not_exists(y_cal, f'{base_path}/y_cal{suffix}')
        save_array_if_not_exists(ids_cal, f'{base_path}/ids_cal{suffix}')
        save_array_if_not_exists(X_test, f'{base_path}/X_test{suffix}')
        save_array_if_not_exists(y_test, f'{base_path}/y_test{suffix}')
        save_array_if_not_exists(ids_test, f'{base_path}/ids_test{suffix}')

        # Upload files to GCS
        def upload_to_gcs(bucket, local_file, remote_file):
            blob = bucket.blob(remote_file)
            blob.upload_from_filename(local_file)
            print(f'Uploaded {local_file} to {remote_file} in GCS.')

        upload_to_gcs(bucket, f'{base_path}/X_train{suffix}', f'{base_path}/X_train{suffix}')
        upload_to_gcs(bucket, f'{base_path}/y_train{suffix}', f'{base_path}/y_train{suffix}')
        upload_to_gcs(bucket, f'{base_path}/ids_train{suffix}', f'{base_path}/ids_train{suffix}')
        upload_to_gcs(bucket, f'{base_path}/X_cal{suffix}', f'{base_path}/X_cal{suffix}')
        upload_to_gcs(bucket, f'{base_path}/y_cal{suffix}', f'{base_path}/y_cal{suffix}')
        upload_to_gcs(bucket, f'{base_path}/ids_cal{suffix}', f'{base_path}/ids_cal{suffix}')
        upload_to_gcs(bucket, f'{base_path}/X_test{suffix}', f'{base_path}/X_test{suffix}')
        upload_to_gcs(bucket, f'{base_path}/y_test{suffix}', f'{base_path}/y_test{suffix}')
        upload_to_gcs(bucket, f'{base_path}/ids_test{suffix}', f'{base_path}/ids_test{suffix}')
            
def get_model_multilabel(num_days, num_features):
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import LSTM, Dense
    model = Sequential()
    model.add(LSTM(64, input_shape=(num_days, num_features)))
    model.add(Dense(3, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy','AUC','Recall', 'Precision'])
    return model

def get_model_multiclass(num_days, num_features):
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import LSTM, Dense
    model = Sequential()
    model.add(LSTM(64, input_shape=(num_days, num_features)))
    model.add(Dense(4, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy','AUC','Recall', 'Precision'])
    return model

def save_model_to_gcs(local_model_path, gcs_model_path):
    import os
    import tensorflow as tf
    from google.cloud import storage
    
    client = storage.Client()
    bucket_name = gcs_model_path.split('/')[2]
    relative_path = '/'.join(gcs_model_path.split('/')[3:])
    bucket = client.get_bucket(bucket_name)
    blob = bucket.blob(relative_path)
    blob.upload_from_filename(local_model_path)
    print(f"Model saved to GCS: {gcs_model_path}")

def load_model_from_gcs(gcs_model_path, local_model_path):
    import os
    import tensorflow as tf
    from google.cloud import storage
    
    client = storage.Client()
    bucket_name = gcs_model_path.split('/')[2]
    relative_path = '/'.join(gcs_model_path.split('/')[3:])
    bucket = client.get_bucket(bucket_name)
    blob = bucket.blob(relative_path)
    blob.download_to_filename(local_model_path)
    print(f"Model downloaded from GCS: {gcs_model_path}")
    return tf.keras.models.load_model(local_model_path)

def train_model(model, X, y, class_weights, X_val, y_val, local_model_path, gcs_model_path):
    from tensorflow.keras.callbacks import ReduceLROnPlateau
    from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    model_checkpoint = ModelCheckpoint(local_model_path, save_best_only=True, monitor='val_loss')

    lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)

    model.fit(X, y, epochs=50, validation_data=(X_val, y_val), 
              class_weight={index: value for index, value in enumerate(class_weights)},
              callbacks=[early_stopping, model_checkpoint, lr_scheduler])
    
    # Save the model to GCS
    save_model_to_gcs(local_model_path, gcs_model_path)
    
    return model

# def train_model(model, X, y, class_weights):
#     model.fit(X, 
#               y, 
#               epochs=5, 
#               class_weight={index: value for index, value in enumerate(class_weights)})
#     return model

def convert_multilabel_to_multiclass(y_multilabel):
    # Define the mapping from multilabel to multiclass
    mapping = {
        (1, 0, 0): 0,
        (1, 1, 0): 1,
        (0, 1, 0): 2,
        (0, 0, 1): 3
    }
    y_multiclass = [mapping[tuple(label)] for label in y_multilabel]
    return np.array(y_multiclass)

def convert_multilabel_to_multiclass_ohe(y_multilabel):
    from tensorflow.keras.utils import to_categorical
    y_multiclass = convert_multilabel_to_multiclass(y_multilabel)
    
    # One-hot encode the multiclass labels
    y_ohe = to_categorical(y_multiclass)
    return np.array(y_ohe)


from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_is_fitted
# Custom wrapper to make the Keras model compatible with scikit-learn and MAPIE/etc.
class KerasMultiLabelWrapper(BaseEstimator, ClassifierMixin):
    def __init__(self, model):
        self.model = model

    def fit(self, X=None, y=None):
        if X and y:                 # call fit() with no parameters if underlying model is already fit
            self.model.fit(X, y, epochs=5, batch_size=32, verbose=2)
        self.classes_ = np.array([0, 1])  # Binary classes
        self.is_fitted_ = True
        return self

    def predict(self, X):
        check_is_fitted(self, 'is_fitted_')
        proba = self.model.predict(X)
        return (proba > 0.5).astype(int)

    def predict_proba(self, X):
        check_is_fitted(self, 'is_fitted_')
        return self.model.predict(X)

# Custom wrapper to make the Keras model compatible with scikit-learn and MAPIE/etc.
class KerasMultiClassWrapper(BaseEstimator, ClassifierMixin):
    def __init__(self, model):
        self.model = model

    def fit(self, X=None, y=None):
        if X and y:                 # call fit() with no parameters if underlying model is already fit
            self.model.fit(X, y, epochs=5, batch_size=32, verbose=2)
        self.classes_ = np.unique(y)
        self.is_fitted_ = True
        return self

    def predict(self, X):
        check_is_fitted(self, 'is_fitted_')
        proba = self.model.predict(X)
        return tf.one_hot(tf.argmax(proba, axis=1), depth=proba.shape[-1]).numpy().astype(int)

    def predict_proba(self, X):
        check_is_fitted(self, 'is_fitted_')
        return self.model.predict(X)

def wrap_model_multilabel(model):
    return KerasMultiLabelWrapper(model=model).fit()

def wrap_model_multiclass(model):
    return KerasMultiClassWrapper(model=model).fit()

def calibrate_multilabel(all_probabilities, all_targets, nonconformity):
    """
    Calibrate nonconformity scores for multilabel classification.

    Parameters
    ----------
    all_probabilities : np.ndarray
        Predicted probabilities for each label. 
    all_targets : np.ndarray
        True class labels.
    nonconformity : function
        Function to compute nonconformity scores.

    Returns
    -------
    lbl_idx_to_alphas_tuple : dict
        Dictionary mapping each label index to its nonconformity scores (negative and positive).
    """
    alphas = np.ones_like(all_probabilities)
    for i in range(all_probabilities.shape[0]):
        for j in range(all_probabilities.shape[-1]):
            alphas[i][j] = nonconformity(all_probabilities[i], j)
            
    # dictionary of alphas for each label/column
    lbl_idx_to_alphas_tuple = {}
    for label_idx in range(alphas.shape[-1]):
        true_labels = all_targets[:, label_idx].astype(bool)
        label_alphas = alphas[:, label_idx]
        
        # get alphas where true class is negative/positive case, respectively
        alpha_neg = 1-label_alphas[~true_labels]
        alpha_pos = label_alphas[true_labels]
        
        lbl_idx_to_alphas_tuple[label_idx] = (alpha_neg, alpha_pos)
 
    return lbl_idx_to_alphas_tuple

def calibrate_multiclass(all_probabilities, all_targets, nonconformity):
    """
    Calibrate nonconformity scores for multiclass classification.

    Parameters
    ----------
    all_probabilities : np.ndarray
        Predicted probabilities for each class.
    all_targets : np.ndarray
        True class labels. Can be 1D array of class indices or 2D one-hot encoded.
    nonconformity : function
        Function to compute nonconformity scores.

    Returns
    -------
    lbl_idx_to_alphas : dict
        Dictionary mapping each class label to its nonconformity scores.
    """
    # Check if all_targets is one-hot encoded and convert if necessary
    if all_targets.ndim == 2:
        all_targets = np.argmax(all_targets, axis=1)
    
    alphas = np.ones_like(all_probabilities)
    for i in range(all_probabilities.shape[0]):
        for j in range(all_probabilities.shape[-1]):
            alphas[i, j] = nonconformity(all_probabilities[i], j)
    
    lbl_idx_to_alphas = {}
    for label_idx in range(alphas.shape[1]):
        true_labels = all_targets == label_idx
        label_alphas = alphas[:, label_idx]
        
        # Get the nonconformity scores for examples where current idx is y_true
        lbl_idx_to_alphas[label_idx] = label_alphas[true_labels]
 
    return lbl_idx_to_alphas

def conformal_prediction_multilabel(pred, alpha_dict, class_conditional, nonconformity):
    n_examples, n_labels = pred.shape
    prediction_results = []
    
    for i in range(n_examples):
        example_results = []
        for lbl in range(n_labels):
            a_neg, a_pos = alpha_dict[lbl]
            p = pred[i, lbl]
            p_binary = [1-p, p]
            
            if class_conditional:
                example_results.append(PredictionClass([
                    ((sum(a_neg >= nonconformity(probs=p_binary, target_idx=0)) + 1) / (len(a_neg) + 1), 0),
                    ((sum(a_pos >= nonconformity(probs=p_binary, target_idx=1)) + 1) / (len(a_pos) + 1), 1)
                ], eps=0.1))
            else:
                a = np.concatenate((a_neg, a_pos))
                example_results.append(PredictionClass([
                    ((sum(a >= nonconformity(p_binary, target_idx=0)) + 1) / (len(a) + 1), 0),
                    ((sum(a >= nonconformity(p_binary, target_idx=1)) + 1) / (len(a) + 1), 1)
                ], eps=0.1))
        
        prediction_results.append(example_results)
    
    return prediction_results

def conformal_prediction_multiclass(pred, alpha_dict, class_conditional, nonconformity):
    """
    Perform conformal prediction for multiclass classification.

    Parameters
    ----------
    pred : np.ndarray
        Predicted probabilities for each class.
    alpha_dict : dict
        Dictionary mapping each class label to its nonconformity scores.
    class_conditional : bool
        Whether to use class-conditional calibration nonconformity scores.
    nonconformity : function
        Function to compute nonconformity scores.

    Returns
    -------
    prediction_results : list
        List of PredictionClass instances for each example.
    """
    n_examples, n_labels = pred.shape
    prediction_results = []

    for i in range(n_examples):
        example_results = []
        for lbl in range(n_labels):
            if class_conditional:
                lbl_alphas = alpha_dict[lbl]
                p_value = (sum(lbl_alphas >= nonconformity(probs=pred[i], target_idx=lbl)) + 1) / (len(lbl_alphas) + 1)
            else:
                all_alphas = np.concatenate(list(alpha_dict.values()))
                p_value = (sum(all_alphas >= nonconformity(probs=pred[i], target_idx=lbl)) + 1) / (len(all_alphas) + 1)
                
            example_results.append((p_value, lbl))

        prediction_results.append(PredictionClass(example_results, eps=0.1))

    return prediction_results

def flatten_results_multilabel(prediction_set):
    return [
        (example_idx, label_idx, prediction)
        for example_idx, example_results in enumerate(prediction_set)
        for label_idx, prediction in enumerate(example_results)
    ]

def flatten_results_multiclass(prediction_set):
    return [
        (example_idx, prediction)
        for example_idx, prediction in enumerate(prediction_set)
    ]

####

def combine_and_flatten_results(run_results):
    combined_results = []

    for run_idx, (flattened_results, y_test, y_pred_probs_test, ids_test) in enumerate(run_results):
        multilabel = True if len(flattened_results[0]) == 3 else False
        
        if multilabel:
            for example_idx, label_idx, prediction in flattened_results:
                combined_results.append({
                    "run": run_idx,
                    "example_idx": example_idx,
                    "label_idx": label_idx,
                    "prediction": prediction,
                    "y_true": y_test[example_idx, label_idx],
                    "y_pred_prob_base_clf": y_pred_probs_test[example_idx, label_idx],
                    "person_id": ids_test[example_idx]
                })
        else:
            # Check if all_targets is one-hot encoded and convert if necessary
            if y_test.ndim == 2:
                y_test = np.argmax(y_test, axis=1)
                
            for example_idx, prediction in flattened_results:
                combined_results.append({
                    "run": run_idx,
                    "example_idx": example_idx,
                    "prediction": prediction,
                    "y_true": y_test[example_idx],
                    "y_pred_prob_base_clf": y_pred_probs_test[example_idx],
                    "person_id": ids_test[example_idx]
                })

    return combined_results

def conformal_pred_and_alpha_str_to_tuple(row):
    import ast
    return ast.literal_eval(row.conformal_pred_and_alpha)

def y_pred_prob_base_clf_str_to_array(row):
    s = row.y_pred_prob_base_clf
    if isinstance(s, float):
        return s
    
    s = s.strip('[')
    s = s.strip(']')
    return np.array([float(p) for p in s.split()])
    
def prediction_p_str_to_tuple_list(row):
    import ast
    return ast.literal_eval(row.prediction_p)

def prediction_p_to_prediction(row):
    if type(row.prediction_p) is str:
        return PredictionClass(p=prediction_p_str_to_tuple_list(row), eps=0.1)
    else:
        return PredictionClass(p=row.prediction_p, eps=0.1)

def calculate_confidence_and_credibility(df):
    df['confidence'] = df.apply((lambda row: row.prediction.confidence()), axis=1)
    df['credibility'] = df.apply((lambda row: row.prediction.credibility()), axis=1)
    return df

def get_underlying_classifier_prediction_multiclass(df):
    return df['y_pred_prob_base_clf'].apply((lambda x: np.argmax(x)))

def get_underlying_classifier_prediction_multilabel(df):
    return df['y_pred_prob_base_clf'].apply((lambda x: int(x >= 0.5)))

def set_allowable_error_and_update_cp(df, alpha):
    def set_eps(pred, eps):
        pred.eps = eps
        return pred
    df['prediction'].apply((lambda x: set_eps(x, alpha)))
    df['conformal_pred_and_alpha'] = df['prediction'].apply((lambda x: (x.classes(alpha), alpha)))

def results_list_to_df(results, get_underlying_classifier_prediction_fn):
    # Convert to a DataFrame for easier analysis
    results_df = pd.DataFrame(combine_and_flatten_results(results))

    results_df['base_clf_pred'] = get_underlying_classifier_prediction_fn(results_df)
    
    # store `prediction.p` to allow re-instantiating PredictionClass upon read from CSV
    results_df['prediction_p'] = results_df['prediction'].apply((lambda x: x.p))
    
    alpha = 0.1
    results_df['conformal_pred_and_alpha'] = results_df['prediction'].apply((lambda x: (x.classes(alpha), alpha)))
    
    # Add confidence and credibility
    results_df = calculate_confidence_and_credibility(results_df)

    # combine w/demographics
    results_df = results_df.merge(demo_df, on='person_id',how='left')
    
    cols = ['run', 'example_idx', 'person_id', 
            'y_true', 'base_clf_pred', 'conformal_pred_and_alpha', 'confidence', 'credibility', 'prediction_p', 
            'gender', 'race', 'ethnicity', 'sex_at_birth', 'age', 'prediction', 'y_pred_prob_base_clf'] 
    
    if results_df.columns.str.contains('label_idx').any():
        cols.insert(2, 'label_idx')
    
    results_df = results_df[cols]
    
    return results_df

def results_list_to_df_multilabel(results):
    return results_list_to_df(results, get_underlying_classifier_prediction_multilabel)

def results_list_to_df_multiclass(results):
    return results_list_to_df(results, get_underlying_classifier_prediction_multiclass)








def results_to_df_mapie_multilabel(results):
    # Flatten and organize data
    flattened_data = []

    for run_idx, (y_true, y_pred_probs_base_clf, person_id, crc, rcps, ltt) in enumerate(results):
        for example_idx in range(y_true.shape[0]):
            for label_idx in range(y_true.shape[1]):
                flattened_data.append({
                    'run': run_idx,
                    'example_idx': example_idx,
                    'label_idx': label_idx,
                    'crc': crc[example_idx, label_idx],
                    'rcps': rcps[example_idx, label_idx],
                    'ltt': ltt[example_idx, label_idx],
                    'y_true': y_true[example_idx, label_idx],
                    'y_pred_prob_base_clf': y_pred_probs_base_clf[example_idx, label_idx],
                    'person_id': person_id[example_idx]
                })

    # Create DataFrame
    return pd.DataFrame(flattened_data)

def results_to_df_mapie_multiclass(results):
    # Flatten and organize data
    flattened_data = []

    for run_idx, (y_true, y_pred_probs_base_clf, person_id, lac, aps, raps, topk) in enumerate(results):
        for example_idx in range(y_true.shape[0]):
#             for label_idx in range(y_true.shape[1]):
            flattened_data.append({
                'run': run_idx,
                'example_idx': example_idx,
#                 'label_idx': label_idx,
                'lac': np.where(lac[example_idx])[0],
                'aps': np.where(aps[example_idx])[0],
                'raps': np.where(raps[example_idx])[0],
                'top_k': np.where(topk[example_idx])[0],
                'y_true': y_true[example_idx],
                'y_pred_prob_base_clf': y_pred_probs_base_clf[example_idx],
                'person_id': person_id[example_idx]
            })

    # Create DataFrame
    return pd.DataFrame(flattened_data)

import re
def results_csv_to_df_mapie(path):
    
    # Function to convert the string back to a list of numbers
    def convert_to_list(string, type_fn):
        # Remove the square brackets, and strip any leading/trailing spaces
        string = string.strip('[]').strip()
        # Replace multiple spaces with a single space
        string = re.sub(r'\s+', ' ', string)
        # Convert the split numbers to type
        return [type_fn(num) for num in string.split()]

    df = pd.read_csv(path)    
    #convert int/float lists back from strings
    for col in df.columns[df.dtypes == 'object']:
        if col == 'y_pred_prob_base_clf':
            df[col] = df[col].apply((lambda x: convert_to_list(x, float)))
            #df[col] = df[col].apply(lambda x: np.array([float(a) for a in x.strip('[]').strip().replace('  ', ' ').split(' ')]))
        else:
            df[col] = df[col].apply((lambda x: convert_to_list(x, int)))
            #df[col] = df[col].apply(lambda x: np.array([int(a) for a in x.strip('[]').strip().replace('  ', ' ').split(' ')]))
    return df

def results_csv_to_df(path):
    df = pd.read_csv(path)
    df['prediction_p'] = df.apply(prediction_p_str_to_tuple_list, axis=1)
    df['prediction'] = df.apply(prediction_p_to_prediction, axis=1)
    df['conformal_pred_and_alpha'] = df.apply(conformal_pred_and_alpha_str_to_tuple, axis=1)
    df['y_pred_prob_base_clf'] = df.apply(y_pred_prob_base_clf_str_to_array, axis=1)
    df = calculate_confidence_and_credibility(df)
    return df

### util start

def get_structure_info(obj, level=0):
    indent = '  ' * level
    if isinstance(obj, dict):
        print(f"{indent}dict with {len(obj)} keys")
        for key, value in obj.items():
            print(f"{indent}  key: {key} -> ", end="")
            get_structure_info(value, level + 1)
    elif isinstance(obj, list):
        print(f"{indent}list of length {len(obj)}")
        if len(obj) > 0:
            get_structure_info(obj[0], level + 1)
    elif isinstance(obj, tuple):
        print(f"{indent}tuple of length {len(obj)}")
        if len(obj) > 0:
            for i in range(len(obj)):
                get_structure_info(obj[i], level + 1)
    elif isinstance(obj, np.ndarray):
        print(f"{indent}ndarray with shape {obj.shape}")
    else:
        print(f"{indent}{type(obj).__name__}")

def compare_code(code1, code2):
    """
    Compare two sets of lines of code and print the differences.

    Parameters:
    code1 (str): The first set of lines of code.
    code2 (str): The second set of lines of code.
    """
    import difflib
    code1_lines = code1.strip().splitlines()
    code2_lines = code2.strip().splitlines()

    diff = difflib.unified_diff(
        code1_lines, code2_lines,
        fromfile='code1', tofile='code2',
        lineterm=''
    )

    print('\n'.join(diff))

### util end


def apply_verdict(row):
    return row['prediction'].verdict(row['y_true'])

# Function to calculate the ratio of true verdicts per demographic group
def calculate_verdict_ratio(df, demographic_col, y_true_condition=None, per_run=False, per_label=False):
    # Filter by y_true condition if specified
    if y_true_condition is not None:
        df = df[df['y_true'] == y_true_condition]

    def calc_ratio(group):
        verdict_true = group[group['verdict'] == True]
        return verdict_true[demographic_col].value_counts() / group[demographic_col].value_counts()

    if per_run and per_label:
        result = df.groupby(['run', 'label_idx']).apply(calc_ratio).unstack(fill_value=0)
    elif per_run:
        result = df.groupby('run').apply(calc_ratio).unstack(fill_value=0)
    elif per_label:
        result = df.groupby('label_idx').apply(calc_ratio).unstack(fill_value=0)
    else:
        result = calc_ratio(df)

    return result

def print_y_true_0_1_verdict_ratios(df, demographic_col='gender'):
    # Run the calculation for y_true == 1
    y_true_1_ratio = calculate_verdict_ratio(df, demographic_col=demographic_col, y_true_condition=1)

    # Run the calculation for y_true == 0
    y_true_0_ratio = calculate_verdict_ratio(df, demographic_col=demographic_col, y_true_condition=0)

    print("\n\n\nY True == 1 Ratio:")
    print(y_true_1_ratio)
    print("\n\n\nY True == 0 Ratio:")
    print(y_true_0_ratio)

def calculate_prediction_set_size_total_count(df, eps=0.1):
    cp_classes = df['prediction'].apply((lambda p: p.classes(eps=eps)))
    return cp_classes.apply(len).sum()

# Function to calculate the total count of predicted classes per group
def calculate_prediction_set_sizes(df, demographic_col, per_run=False, per_label=False, allowable_error_rate=0.1):
    eps=allowable_error_rate
    def get_prediction_set_size(row):
        return len(row['prediction'].classes(eps=eps))

    # Calculate the size of the prediction set for each row
    col = f'prediction_set_size__at_eps_{str(eps)}'
    df[col] = df.apply(get_prediction_set_size, axis=1)

    if per_run and per_label:
        result = (df.groupby(['run', 'label_idx', demographic_col])[col].sum() / 
                  df[['run', 'label_idx', demographic_col]].groupby(['run', 'label_idx']).value_counts()).unstack(fill_value=0)
    elif per_run:
        result = (df.groupby(['run', demographic_col])[col].sum() / 
                  df[['run', demographic_col]].groupby('run').value_counts()).unstack(fill_value=0)
    elif per_label:
        result = (df.groupby(['label_idx', demographic_col])[col].sum() / 
                  df[['label_idx', demographic_col]].groupby('label_idx').value_counts()).unstack(fill_value=0)
    else:
        result = df.groupby(demographic_col)[col].sum() / df[demographic_col].value_counts()

    return result    


#### ~/vigilant-computing-machine

def sort_reindex(df_pred, col=['confidence','credibility']):
    '''
    Sort df by the provided column(s), update index to sorted order

    Parameters
    ----------
    df_pred : 'pandas.DataFrame'
        A dataframe to be sorted by the provided columns.
    col : list, optional
        Column(s) to sort from left to right. 
        The default is ['confidence','credibility'].

    Returns
    -------
    'pandas.DataFrame'
        The sorted and reindexed dataframe.

    '''
    if isinstance(col, str):
        return df_pred.sort_values(by=col, ascending=False, kind='mergesort').reset_index(drop=True)
    if isinstance(col, list):
        return df_pred.sort_values(by=col, ascending=[False]*len(col), kind='mergesort').reset_index(drop=True)

def plot_confidence_credibility_by_index(df_pred, title='', ax=None, alpha=1.0, legend=True):
    '''
    Plot 'confidence' and 'credibility' columns of dataframe by index

    Parameters
    ----------
    df_pred : 'pandas.DataFrame'
        dataframe to plot.
    title : string, optional
        plot title. The default is ''.
    ax : 'matplotlib.pyplot.axis', optional
        axis to plot on. The default is None.
    alpha : float, optional
        alpha value of lines. The default is 1.0.
    legend : bool, optional
        display legend in graph. The default is True

    Returns
    -------
    None.

    '''
    df_pred.plot(y=['confidence', 'credibility'], use_index=True, title=title, ax=ax, alpha=alpha, legend=legend)
    ax.set_ylabel('Score')
    
def plot_conf_cred(df_pred, title='', scale=True, ax=None, alpha=1.0, legend=True):
    '''
    Plot df sorted and indexed by 'confidence' and 'credibility'

    Parameters
    ----------
    df_pred : 'pandas.DataFrame'
        dataframe to plot.
    title : string, optional
        plot title. The default is ''.
    scale : bool, optional
        scale plot. The default is True.
    ax : 'matplotlib.pyplot.axis', optional
        axis to plot on. The default is None.
    alpha : float, optional
        alpha value of lines. The default is 1.0.
    legend : bool, optional
        display legend in graph. The default is True

    Returns
    -------
    None.

    '''
    df = sort_reindex(df_pred, ['confidence', 'credibility'])
    if scale:
        from sklearn.preprocessing import MinMaxScaler
        scaler = MinMaxScaler()
        df[['confidence', 'credibility']] = scaler.fit_transform(df[['confidence', 'credibility']])
    plot_confidence_credibility_by_index(df, title=title, ax=ax, alpha=alpha, legend=legend)

def plot_conf_cred_2(df1, df2, title1='Class-Conditional', title2='Non-Class-Conditional'):
    import matplotlib.pyplot as plt
    fig, axs = plt.subplots(2, 1, figsize=(12, 15))
    
    # Plot class-conditional data
    axs[0].vlines(x=len(df1) * .9, ymin=0, ymax=1, linestyle='--', label='LDR', color='g', linewidth=1)
    plot_conf_cred(df1, title=title1, alpha=0.4, ax=axs[0], scale=False)
    axs[0].set_title(title1)
    axs[0].set_ylabel('Score')
    axs[0].legend()
    
    # Plot non-class-conditional data
    axs[1].vlines(x=len(df2) * .9, ymin=0, ymax=1, linestyle='--', label='LDR', color='g', linewidth=1)
    plot_conf_cred(df2, title=title2, alpha=0.4, ax=axs[1], scale=False)
    axs[1].set_title(title2)
    axs[1].set_ylabel('Score')
    axs[1].legend()
    
    plt.tight_layout()
    plt.show()
    
#### ~/vigilant-computing-machine

def is_y_true_in_cp(df):
    return df.apply((lambda row: row['y_true'] in row['conformal_pred_and_alpha'][0]), axis=1)

def calculate_y_true_in_cp_percent(df, per_run=False):
    if per_run:
        return df.groupby('run').apply(calculate_y_true_in_cp_percent)
    else:
        return is_y_true_in_cp(df).sum() / len(df)

def is_y_true_same_as_base_clf_pred(df):
    return df.apply((lambda row: row['y_true'] == row['base_clf_pred']), axis=1) 

def calculate_y_true_same_as_base_clf_pred_percent(df, per_run=False):
    if per_run:
        return df.groupby('run').apply(calculate_y_true_same_as_base_clf_pred_percent)
    else:
        return is_y_true_same_as_base_clf_pred(df).sum() / len(df)    
    
# TODO per label/run/etc. grouping and conditions

def print_base_clf_report_multiclass(df, labels=None):
    from sklearn.metrics import classification_report

    y = df['y_true'].values
    base_clf_pred = df['base_clf_pred'].values
    print(classification_report(y, base_clf_pred, target_names=labels))

def print_base_clf_report_multilabel(df, labels=None):
    from sklearn.metrics import classification_report

    ys = []
    preds = []
    for grp, grp_df in df.groupby(['run','example_idx']):
        ys.append(grp_df['y_true'].values)
        preds.append(grp_df['base_clf_pred'].values)

    print(classification_report(np.array(ys), np.array(preds), target_names=labels))
    del ys
    del preds

def print_base_clf_report_multilabel_binary(df, labels=None):
    from sklearn.metrics import classification_report

    for grp, grp_df in results_name_df_tuples[10][1].groupby('label_idx'):
        print(f'-----------------\nLabel: {grp if labels is None else labels[grp]}'+
              f'  - present {grp_df.y_true.sum() / len(grp_df) * 100:.1f}% of patients\n-----------------')
        print_base_clf_report_multiclass(grp_df, ['Disorder  Absent', 'Disorder Present'])

        
def get_exp_rep_table(df, demographic_col='gender'):
    sample_size = len(df)

    n = df[demographic_col].value_counts().sort_index()
    nr = n.values.sum()
    d = pd.DataFrame()
    d[f'all_{demographic_col}_n'] = n
    d[f'all_{demographic_col}'] = n / nr

    lc = get_low_confidence_predictions(df, 10)
    n = lc[demographic_col].value_counts().sort_index()
    nr = n.values.sum()
    d[f'low_{demographic_col}_n'] = n
    d[f'low_{demographic_col}'] = n / nr
    d = d.fillna(0)
    d[f'low_{demographic_col}_n'] = d[f'low_{demographic_col}_n'].astype(int)
    return d

def get_low_confidence_predictions(df, percentile=10):
    #check valid range
    assert 0 < percentile and percentile < 100, 'percentile outside valid range'
    if percentile < 1:
        percentile = percentile * 100
    #integer math rounds down, preventing rounding up beyond bounds for small collections (though .head() handles)
    #open to reasons to round up
    num_to_return = len(df) // percentile
    #when predictions with the same confidence value span the percentile boundary,
    #credibility will be used as the secondary sort criteria
    #when predictions with the same confidence and credibility span the percentile boundary,
    #predictions will be returned based upon their index values in the provided DataFrame
    df = df.sort_values(by=['confidence', 'credibility'], kind='mergesort')
    return df.iloc[:num_to_return, :]

def get_low_credibility_predictions(df, percentile=10):
    #check valid range
    assert 0 < percentile and percentile < 100, 'percentile outside valid range'
    if percentile < 1:
        percentile = percentile * 100
    #integer math rounds down, preventing rounding up beyond bounds for small collections (though .head() handles)
    #open to reasons to round up
    num_to_return = len(df) // percentile
    #when predictions with the same credibility value span the percentile boundary,
    #confidence will be used as the secondary sort criteria
    #when predictions with the same confidence and credibility span the percentile boundary,
    #predictions will be returned based upon their index values in the provided DataFrame
    df = df.sort_values(by=['credibility', 'confidence'], kind='mergesort')
    return df.iloc[:num_to_return, :]

def get_experiments_max_low_confidence(experiments):
    return [get_low_confidence_predictions(exp.df[0], percentile=10).confidence.max() for exp in experiments]

def get_experiments_max_low_credibility(experiments):
    return [get_low_credibility_predictions(exp.df[0], percentile=10).credibility.max() for exp in experiments]

def get_experiments_median_low_credibility(experiments):
    return [get_low_credibility_predictions(exp.df[0], percentile=10).credibility.median() for exp in experiments]

def get_incorrect_predictions(df):
    return df[~df['verdict']]

def get_empty_predictions_mask(df):
    return df.classes.apply((lambda x: len(x) == 0))

def get_single_predictions_mask(df):
    return df.classes.apply((lambda x: len(x) == 1))

def get_multiple_predictions_mask(df):
    return df.classes.apply((lambda x: len(x) > 1))

def df_col_to_int_if_float_is_integer_all(df, col_name):
    '''
    Coerce df column to int if all are float where float.is_integer == True

    Parameters
    ----------
    df : 'pandas.DataFrame'
        dataframe containing column to potentially coerce to int.
    col_name : string
        name of column.

    Returns
    -------
    None.

    '''
    # if all instances in this column are integers in float representation
    if df.loc[:,col_name].apply(float.is_integer).all():
        # convert all instances in this column to integer
        df.loc[:,col_name] = df.loc[:,col_name].astype(int)

def df_pred_empty_mean(df, pred_col_name='classes'):
    '''
    Calculate the proportion of predictions 
    (generally, could be any column containing a type that can have len() applied) 
    that are empty.

    Parameters
    ----------
    df : 'pandas.DataFrame'
        dataframe containing predictions.
    pred_col_name : string, optional
        column name. The default is 'classes'.

    Returns
    -------
    float
        proportion of empty entries.

    '''
    return df[pred_col_name].apply(lambda x: len(x) < 1).mean()

def df_pred_singleton_mean(df, pred_col_name='classes'):
    '''
    Calculate the proportion of predictions 
    (generally, could be any column containing a type that can have len() applied) 
    that are of length 1.

    Parameters
    ----------
    df : 'pandas.DataFrame'
        dataframe containing predictions.
    pred_col_name : string, optional
        column name. The default is 'classes'.

    Returns
    -------
    float
        proportion of singleton entries.

    '''
    return df[pred_col_name].apply(lambda x: len(x)==1).mean()

def df_pred_singleton_correct_mean(df, pred_col_name='classes', verdict_col_name='verdict'):
    '''
    Calculate the proportion of predictions that are of length 1 and contain 
    the correct class.

    Parameters
    ----------
    df : 'pandas.DataFrame'
        dataframe containing predictions and verdicts.
    pred_col_name : string, optional
        prediction column name. The default is 'classes'.
    verdict_col_name : string, optional
        verdict column name. The default is 'verdict'.


    Returns
    -------
    float
        proportion of singleton entries that correspond to the correct class.

    '''
    single_and_correct = (df[pred_col_name].apply(lambda x: len(x)==1) & df[verdict_col_name])
    return sum(single_and_correct) / len(df)

def df_pred_multiple_mean(df, pred_col_name='classes'):
    '''
    

    Calculate the proportion of predictions 
    (generally, could be any column containing a type that can have len() applied) 
    that are of length greater than 1.

    Parameters
    ----------
    df : 'pandas.DataFrame'
        dataframe containing predictions.
    pred_col_name : string, optional
        column name. The default is 'classes'.

    Returns
    -------
    float
        proportion of multiple entries.

    '''
    return df[pred_col_name].apply(lambda x: len(x) > 1).mean()

## `run_experiment`

In [ ]:
import os

def run_experiment(daily_df,
                   demographic_df,
                   label_df,
                   get_model_fn, 
                   train_model_fn,
                   wrap_model_fn,
                   nonconformity_fn,
                   calibrate_fn,
                   conformal_prediction_fn,
                   class_conditional,
                   flatten_results_fn,
                   times=10, 
                   num_days=10,
                   model_name_suffix=''):
    
    use_demographics = demographic_df is not None
    
    run_flattenedResult_yTrue_yPredProbs_personId = []
    
    my_bucket = os.getenv('WORKSPACE_BUCKET')
    
    for i in range(times):
        
        # reproducible experimental variability
        set_seeds(offset=i)
        
         # Determine the model type (multilabel or multiclass) based on the get_model_fn
        model_type = 'multilabel' if get_model_fn.__name__.startswith('get_model_multilabel') else 'multiclass'
        model_type += '_with_demo' if use_demographics else '_wout_demo'
        
        # Define the base path for the experiment
        base = f'data/experiments/{i}'
        base_path = f'{my_bucket}/{base}'
        gcs_model_path = f'{base_path}/{model_type}/best_model_{i}_{str(num_days)}_day_seqs'+model_name_suffix+'.h5'
        local_model_path = f'{base}/{model_type}/best_model_{i}_{str(num_days)}_day_seqs'+model_name_suffix+'.h5'
        
        # Create local directories if they don't exist
        os.makedirs(os.path.dirname(local_model_path), exist_ok=True)
        
        # load this experiment configurations data, if exists
        exists, \
        X_train, X_cal, X_test, \
        y_train, y_cal, y_test, \
        ids_train, ids_cal, ids_test = does_this_experiments_input_data_exist(run_num=i, 
                                                                              num_days=num_days, 
                                                                              with_demographics=use_demographics)
        
        # generate this experiment configurations data, if not exists
        if not exists:            
            df = get_random_contiguous_chunks_with_no_missing_data(daily_df, 
                                                                   chunk_size=num_days, 
                                                                   max_attempts=10)

            if use_demographics:
                df, categorical_cols = merge_demographics_and_ohe_categorical_columns(df, demographic_df)
                
            excl_scaling_cols = [*categorical_cols] if use_demographics else []
            
            (X_train, y_train, ids_train), \
            (X_cal, y_cal, ids_cal), \
            (X_test, y_test, ids_test) = get_scaled_reshaped_train_calibrate_test(df, 
                                                                                  label_df, 
                                                                                  num_days=num_days,
                                                                                  random_offset=i, 
                                                                                  exclude_scale_cols=excl_scaling_cols)

            # write this experiment configurations data
            write_this_experiments_input_data_if_not_exists(i, 
                                                            num_days, 
                                                            use_demographics, 
                                                            X_train, y_train, ids_train, 
                                                            X_cal, y_cal, ids_cal, 
                                                            X_test, y_test, ids_test)
        
        if model_type.startswith('multiclass'):
            y_train, y_cal, y_test = convert_multilabel_to_multiclass_ohe(y_train), convert_multilabel_to_multiclass_ohe(y_cal), convert_multilabel_to_multiclass_ohe(y_test)

        # Define class weights
        label_counts = y_train.sum(axis=0)
        print(f'\n\nLabel counts (run {i}): {label_counts}\n')
            
        # Check if the model already exists
        if os.path.exists(local_model_path):
            import tensorflow as tf
            print(f"Loading existing model from {local_model_path}")
            mod = tf.keras.models.load_model(local_model_path)
        else:
            
            if storage.Client().bucket(gcs_model_path.split('/')[2]).blob('/'.join(gcs_model_path.split('/')[3:])).exists():
                print(f"Downloading existing model from GCS: {gcs_model_path}")
                mod = load_model_from_gcs(gcs_model_path, local_model_path)
            else:
                # Get sequence shape
                num_days = X_train.shape[1]
                num_features = X_train.shape[2]

                print(f'num days: {num_days}')
                print(f'num features: {num_features}')
                
                # Define class weights
                import tensorflow as tf
                class_weights = tf.constant(label_counts.min()/label_counts, dtype=tf.float32)
                  ## also tried tf.constant(label_counts.max()/label_counts, dtype=tf.float32), marginal differences observed
                
                # sanity check loss weighting
                print(f'\n\nLabel counts (run {i}): {label_counts}\n'+
                      f'Class Weights: {str(class_weights.numpy())}')
                
                # Get model
                mod = get_model_fn(num_days, num_features)
                
                # Train the model and save it to GCS
                mod = train_model_fn(mod, 
                                     X=X_train, 
                                     y=y_train, 
                                     class_weights=class_weights, 
                                     X_val=X_cal, 
                                     y_val=y_cal,
                                     local_model_path=local_model_path, 
                                     gcs_model_path=gcs_model_path)
            
        # wrap
        mod = wrap_model_fn(mod)

        # predict (calibration)
        pred_probs_cal = mod.predict_proba(X_cal)

        # get nonconformity scores
        lbl_idx_to_alphas = calibrate_fn(pred_probs_cal, y_cal, nonconformity_fn)
        
        # predict (test)
        pred_probs_test = mod.predict_proba(X_test)
        
        # Conformal Prediction sets
        prediction_sets = conformal_prediction_fn(pred_probs_test, 
                                                  lbl_idx_to_alphas, 
                                                  class_conditional, 
                                                  nonconformity_fn)
        
        # Flatten the results
        flattened_results = flatten_results_fn(prediction_sets)
        
        run_flattenedResult_yTrue_yPredProbs_personId.append([flattened_results, y_test, pred_probs_test, ids_test])
        
    return run_flattenedResult_yTrue_yPredProbs_personId

## `run_experiment_mapie`

In [ ]:
def run_experiment_mapie(daily_df,
                         demographic_df,
                         label_df,
                         get_model_fn, 
                         train_model_fn,
                         wrap_model_fn,
                         times=10, 
                         num_days=10):
    
    use_demographics = demographic_df is not None
    
    results = []
    
    my_bucket = os.getenv('WORKSPACE_BUCKET')
    
    for i in range(times):
        
        # reproducible experimental variability
        set_seeds(offset=i)
        
         # Determine the model type (multilabel or multiclass) based on the get_model_fn
        model_type = 'multilabel' if get_model_fn.__name__ == 'get_model_multilabel' else 'multiclass'
        model_type += '_with_demo' if use_demographics else '_wout_demo'
        
        # Define the base path for the experiment
        base = f'data/experiments/{i}'
        base_path = f'{my_bucket}/{base}'
        gcs_model_path = f'{base_path}/{model_type}/best_model_{i}_{str(num_days)}_day_seqs.h5'
        local_model_path = f'{base}/{model_type}/best_model_{i}_{str(num_days)}_day_seqs.h5'
        
        # Create local directories if they don't exist
        os.makedirs(os.path.dirname(local_model_path), exist_ok=True)
        
        # load this experiment configurations data, if exists
        exists, \
        X_train, X_cal, X_test, \
        y_train, y_cal, y_test, \
        ids_train, ids_cal, ids_test = does_this_experiments_input_data_exist(run_num=i, 
                                                                              num_days=num_days, 
                                                                              with_demographics=use_demographics)
        
        # generate this experiment configurations data, if not exists
        if not exists:            
            df = get_random_contiguous_chunks_with_no_missing_data(daily_df, 
                                                                   chunk_size=num_days, 
                                                                   max_attempts=10)

            if use_demographics:
                df, categorical_cols = merge_demographics_and_ohe_categorical_columns(df, demographic_df)
                
            excl_scaling_cols = [*categorical_cols] if use_demographics else []
            
            (X_train, y_train, ids_train), \
            (X_cal, y_cal, ids_cal), \
            (X_test, y_test, ids_test) = get_scaled_reshaped_train_calibrate_test(df, 
                                                                                  label_df, 
                                                                                  num_days=num_days,
                                                                                  random_offset=i, 
                                                                                  exclude_scale_cols=excl_scaling_cols)

            # write this experiment configurations data
            write_this_experiments_input_data_if_not_exists(i, 
                                                            num_days, 
                                                            use_demographics, 
                                                            X_train, y_train, ids_train, 
                                                            X_cal, y_cal, ids_cal, 
                                                            X_test, y_test, ids_test)
        is_multilabel = True
        if model_type.startswith('multiclass'):
            y_train, y_cal, y_test = convert_multilabel_to_multiclass(y_train), convert_multilabel_to_multiclass(y_cal), convert_multilabel_to_multiclass(y_test)
            is_multilabel = False

        # Define class weights
        label_counts = y_train.sum(axis=0)
        print(f'\n\nLabel counts (run {i}): {label_counts}\n')
            
        # Check if the model already exists
        if os.path.exists(local_model_path):
            import tensorflow as tf
            print(f"Loading existing model from {local_model_path}")
            mod = tf.keras.models.load_model(local_model_path)
        else:
            
            if storage.Client().bucket(gcs_model_path.split('/')[2]).blob('/'.join(gcs_model_path.split('/')[3:])).exists():
                print(f"Downloading existing model from GCS: {gcs_model_path}")
                mod = load_model_from_gcs(gcs_model_path, local_model_path)
            else:
                # Get sequence shape
                num_days = X_train.shape[1]
                num_features = X_train.shape[2]

                print(f'num days: {num_days}')
                print(f'num features: {num_features}')
                
                # Get model
                mod = get_model_fn(num_days, num_features)
                
                # Define class weights
                import tensorflow as tf
                class_weights = tf.constant(label_counts.min()/label_counts, dtype=tf.float32)
                
                # Train the model and save it to GCS
                mod = train_model_fn(mod, X=X_train, y=y_train, class_weights=class_weights, X_val=X_cal, y_val=y_cal, local_model_path=local_model_path, gcs_model_path=gcs_model_path)
            
        # wrap
        mod = wrap_model_fn(mod)

        if is_multilabel:
            # Wrap the model with MapieMultiLabelClassifier
            #
            #     References
            #     ----------
            #     [1] Lihua Lei Jitendra Malik Stephen Bates, Anastasios Angelopoulos
            #     and Michael I. Jordan. Distribution-free, risk-controlling prediction
            #     sets. CoRR, abs/2101.02703, 2021.
            #     URL https://arxiv.org/abs/2101.02703.39

            #     [2] Angelopoulos, Anastasios N., Stephen, Bates, Adam, Fisch, Lihua,
            #     Lei, and Tal, Schuster. "Conformal Risk Control." (2022).

            #     [3] Angelopoulos, A. N., Bates, S., Candès, E. J., Jordan,
            #     M. I., & Lei, L. (2021). Learn then test:
            #     "Calibrating predictive algorithms to achieve risk control".
            #
            #
            # valid_methods_by_metric_ = {
            #         "precision": ["ltt"],
            #         "recall": ["rcps", "crc"]
            #     }
            #
            # method = 
            #          'crc'  (Conformal Risk Control) 
            #          'rcps' (Risk-Controlling Prediction Sets) 
            #          'ltt'  (Learn-Then-Test)
            #

            # Conformal Risk Control
            crc = MapieMultiLabelClassifier(estimator=mod, 
                                            metric_control='recall',        # 'recall' -> avoid missing conditions
                                            method='crc',                   # recall='crc'/'rcps'
                                            n_jobs=-1,                      # default 1 => easier debug; -1 for all CPUs
                                            verbose=2,
                                            random_state=42+i)

            # Risk-Controlled Prediction Sets
            rcps = MapieMultiLabelClassifier(estimator=mod, 
                                             metric_control='recall',        # 'recall' -> avoid missing conditions
                                             method='rcps',                  # recall='crc'/'rcps'
                                             n_jobs=-1,                      # default 1 => easier debug; -1 for all CPUs
                                             verbose=2,
                                             random_state=42+i)

            # Learn-Then-Test
            ltt = MapieMultiLabelClassifier(estimator=mod, 
                                            metric_control='precision',     # 'precision' -> avoid falsely predicting conditions
                                            method='ltt',                   # precision='ltt'
                                            n_jobs=-1,                      # default 1 => easier debug; -1 for all CPUs
                                            verbose=2,
                                            random_state=42+i)

            # calibrate conformal methods
            crc = crc.partial_fit(X_cal, y_cal, _refit=False)
            rcps = rcps.partial_fit(X_cal, y_cal, _refit=False)
            ltt = ltt.partial_fit(X_cal, y_cal, _refit=False)


            # set acceptable level(s) of error for crc/rcps
            error_rates = [0.1]

            # the level of certainty at which we compute the Upper Confidence Bound of the average risk.
            # Lower `delta` produce larger (more conservative) prediction sets.
            #
            delta = 0.01      # 99% certainty distinguishes rcps from crc more markedly than 90%

            # Make predictions
            y_pred, y_pis_crc = crc.predict(X_test, 
                                            alpha=error_rates)

            y_pred, y_pis_rcps = rcps.predict(X_test, 
                                              alpha=error_rates, 
                                              delta=delta)

            y_pred, y_pis_ltt = ltt.predict(X_test,
                                            alpha=[0.5],       # arbitrary user-specified error ~ be right more often than not
                                            delta=delta)

            # get prediction probabilites of underlying classifier
            pred_probs_test = mod.predict_proba(X_test)

            # squeeze per only defined single error rate
            y_crc, y_rcps, y_ltt = y_pis_crc.squeeze(), y_pis_rcps.squeeze(), y_pis_ltt.squeeze()

            run_yTrue_yPredProbs_personIds_crc_rcps_ltt = (y_test, 
                                                           pred_probs_test, 
                                                           ids_test,
                                                           y_crc,
                                                           y_rcps,
                                                           y_ltt)
            
            results.append(run_yTrue_yPredProbs_personIds_crc_rcps_ltt)
            
        else:
            # Wrap the model with MapieClassifier
            #
            #         References
            #     ----------
            #     [1] Mauricio Sadinle, Jing Lei, and Larry Wasserman.
            #     "Least Ambiguous Set-Valued Classifiers with Bounded Error Levels.",
            #     Journal of the American Statistical Association, 114, 2019.
            #
            #     [2] Yaniv Romano, Matteo Sesia and Emmanuel J. Candès.
            #     "Classification with Valid and Adaptive Coverage."
            #     NeurIPS 202 (spotlight) 2020.
            #
            #     [3] Anastasios Nikolas Angelopoulos, Stephen Bates, Michael Jordan
            #     and Jitendra Malik.
            #     "Uncertainty Sets for Image Classifiers using Conformal Prediction."
            #     International Conference on Learning Representations 2021.
            
            mod.classes_ = np.unique(np.concatenate([y_train, y_cal, y_test]))
            
            
            methods = ["lac", "aps", "raps", "top_k"]
            mapie, y_pred_mapie, y_ps_mapie = {}, {}, {}
            error_rates = [0.1]
            for method in methods:
                mapie[method] = MapieClassifier(
                    estimator=mod,
                    method=method,
                    cv="prefit",
                    random_state=42+i,
                )
                mapie[method].fit(X_cal, y_cal)
                y_pred_mapie[method], y_ps_mapie[method] = \
                    mapie[method].predict(X_test, 
                                          alpha=error_rates, 
                                          include_last_label=True   # per coverage guarantee [3]
                                         )
            
            # get prediction probabilites of underlying classifier
            pred_probs_test = mod.predict_proba(X_test)

            # squeeze per only defined single error rate
            y_lac, y_aps, y_raps, y_topk = (y_ps_mapie['lac'].squeeze(), 
                                            y_ps_mapie['aps'].squeeze(), 
                                            y_ps_mapie['raps'].squeeze(),
                                            y_ps_mapie['top_k'].squeeze())
            
            
            
            run_yTrue_yPredProbs_personIds_lac_aps_raps_topk = (y_test, 
                                                                pred_probs_test, 
                                                                ids_test,
                                                                y_lac,
                                                                y_aps,
                                                                y_raps,
                                                                y_topk)
            
            results.append(run_yTrue_yPredProbs_personIds_lac_aps_raps_topk)

        
    return results



# Read in Data

In [ ]:
prefix = f'{my_bucket}/data/dfs/'

# Read CSVs
demo_df = pd.read_csv(f"{prefix}demographics_df.csv")
daily_df = pd.read_csv(f"{prefix}daily_df_v2_prepped.csv")
labels = pd.read_csv(f"{prefix}daily_df_labels_v2.csv", index_col=0)

## Number of Individuals Before Per-Run Fetch of 10-Day Sequences

In [ ]:
labels.shape[0]

13835

## Percent - Multi-Label Imbalance

In [ ]:
temp = labels.sum(axis=0)/labels.shape[0]*100
for i, v in zip(temp.index,temp):
    print(f'{i:<20} {v:.2f}%')

```
Anxiety disorder     16.62%
Depressive disorder  15.73%
No disorder          78.29%
```

## Percent - Multi-Label/Class With Both Anxiety and Depression

In [ ]:
temp = labels[(labels['Anxiety disorder'] == 1) & (labels['Depressive disorder'] == 1)].shape[0]/labels.shape[0]*100
print(f'{temp:.2f}%')

10.63%

### Percent - Multi-Class

In [ ]:
temp = labels[(labels['Anxiety disorder'] == 0) & (labels['Depressive disorder'] == 1)].shape[0]/labels.shape[0]*100
print(f'{temp:.2f}%')

5.10%

In [ ]:
temp = labels[(labels['Anxiety disorder'] == 1) & (labels['Depressive disorder'] == 0)].shape[0]/labels.shape[0]*100
print(f'{temp:.2f}%')

5.98%

In [ ]:
temp = labels[(labels['Anxiety disorder'] == 0) & (labels['Depressive disorder'] == 0)].shape[0]/labels.shape[0]*100
print(f'{temp:.2f}%')

78.29%

# MAPIE

## Import MAPIE (Model Agnostic Prediction Interval Estimator)

In [ ]:
#clone MAPIE repo
!git clone https://github.com/scikit-learn-contrib/MAPIE

    
#add MAPIE to PYTHONPATH so python can find MAPIE module
import sys

pwd = !pwd
sys.path.append(f'{pwd[0]}/MAPIE')

#imports
from mapie.multi_label_classification import MapieMultiLabelClassifier
from mapie.classification import MapieClassifier

## Run Experiment - Multi-Label (`crc`, `rcps`, `ltt`)

### Include Demographic Covariates

In [ ]:
results = \
    run_experiment_mapie(daily_df=daily_df,
                         demographic_df=demo_df,
                         label_df=labels,
                         get_model_fn=get_model_multilabel, 
                         train_model_fn=train_model,
                         wrap_model_fn=wrap_model_multilabel,
                         times=100,
                         num_days=10)

In [ ]:
df = results_to_df_mapie_multilabel(results)
df

###### Save Results to CSV

In [ ]:
df.to_csv(f'{prefix}results_df__multi-label__mapie_100x__per_run_timespans__no_leak__w_demographic_covariates.csv', index=False)
del df

### Exclude Demographic Covariates

In [ ]:
results = \
    run_experiment_mapie(daily_df=daily_df,
                         demographic_df=None,
                         label_df=labels,
                         get_model_fn=get_model_multilabel, 
                         train_model_fn=train_model,
                         wrap_model_fn=wrap_model_multilabel,
                         times=100,
                         num_days=10)

In [ ]:
df = results_to_df_mapie_multilabel(results)
df

###### Save Results to CSV

In [ ]:
df.to_csv(f'{prefix}results_df__multi-label__mapie_100x__per_run_timespans__no_leak__wout_demographic_covariates.csv', index=False)
del df

# MAPIE - Base vs Conformal

## Read in Results

In [ ]:
prefix = f'{my_bucket}/data/dfs/'

df_ml_wd = results_csv_to_df_mapie(f'{prefix}results_df__multi-label__mapie_100x__per_run_timespans__no_leak__w_demographic_covariates.csv')
df_ml_wo = results_csv_to_df_mapie(f'{prefix}results_df__multi-label__mapie_100x__per_run_timespans__no_leak__wout_demographic_covariates.csv')

## Multi-Label - MAPIE - Get % Recall/Precision Base vs CP per Run 

### Aggregate Multi-Label - Row per Person

In [ ]:
# aggregate up to one row per person per run
def aggregate_ml_to_single_row_per_example(df):
    # Convert boolean columns to integers
    df[['crc', 'rcps', 'ltt']] = df[['crc', 'rcps', 'ltt']].astype(int)
    
    # Group by 'run' and 'example_idx', then aggregate relevant columns
    df_grouped = df.groupby(['run', 'example_idx']).agg({
        'crc': list,
        'rcps': list,
        'ltt': list,
        'y_true': list,
        'y_pred_prob_base_clf': list,
        'person_id': set
    }).reset_index()

    # Convert the lists to numpy arrays
    df_grouped['crc'] = df_grouped['crc'].apply(np.array)
    df_grouped['rcps'] = df_grouped['rcps'].apply(np.array)
    df_grouped['ltt'] = df_grouped['ltt'].apply(np.array)
    df_grouped['y_true'] = df_grouped['y_true'].apply(np.array)
    df_grouped['y_pred_prob_base_clf'] = df_grouped['y_pred_prob_base_clf'].apply(np.array)
    df_grouped['person_id'] = df_grouped['person_id'].apply(lambda x: x.pop())
    
    return df_grouped

### Metrics Multi-Label

- [`base`,`crc`,`rcps`,`ltt`]
  - [`recall`,`precision`]
    - [`micro`,`macro`,`weighted`]
  - `avg_prediction_set_size`
  - `total_false_positives`
  - `total_false_negatives`

In [ ]:
from sklearn.metrics import precision_score, recall_score

def calculate_metrics_with_base(group):
    metrics = {}
    
    # Convert prediction probabilities to binary predictions using a threshold of 0.5
    base_predictions = (np.vstack(group['y_pred_prob_base_clf']) >= 0.5).astype(int)
    
    for method in ['crc', 'rcps', 'ltt', 'base']:
        if method == 'base':
            method_predictions = base_predictions
        else:
            method_predictions = np.vstack(group[method])
        
        y_true = np.vstack(group['y_true'])
        
        # Calculate precision and recall for micro, macro, and weighted averages
        for avg in ['micro', 'macro', 'weighted']:
            precision = precision_score(y_true, method_predictions, average=avg, zero_division=0)
            recall = recall_score(y_true, method_predictions, average=avg)
            metrics[f'{method}_precision_{avg}'] = precision
            metrics[f'{method}_recall_{avg}'] = recall
        
        # Calculate the average prediction set size
        avg_pred_set_size = method_predictions.sum(axis=1).mean()
        metrics[f'{method}_avg_prediction_set_size'] = avg_pred_set_size
        
        # Calculate total false positives and total false negatives
        false_positives = ((method_predictions == 1) & (y_true == 0)).sum()
        false_negatives = ((method_predictions == 0) & (y_true == 1)).sum()
        
        metrics[f'{method}_total_false_positives'] = false_positives
        metrics[f'{method}_total_false_negatives'] = false_negatives
    
    # Reorder the metrics
    ordered_metrics = {}
    for metric in ['recall', 'precision']:
        for avg in ['micro', 'macro', 'weighted']:
            for method in ['base', 'crc', 'rcps', 'ltt']:
                ordered_metrics[f'{method}_{metric}_{avg}'] = metrics[f'{method}_{metric}_{avg}']
    
    for method in ['base', 'crc', 'rcps', 'ltt']:
        ordered_metrics[f'{method}_avg_prediction_set_size'] = metrics[f'{method}_avg_prediction_set_size']
        ordered_metrics[f'{method}_total_false_positives'] = metrics[f'{method}_total_false_positives']
        ordered_metrics[f'{method}_total_false_negatives'] = metrics[f'{method}_total_false_negatives']
        
    return pd.Series(ordered_metrics)


### Print Metrics (CSV format)

In [ ]:
def print_df_in_csv_format(df):
    import sys

    # Create a string from the column names with no spaces between them
    header = ','.join(df.columns)

    # Print the header
    print(header)

    # Print each row in thxe DataFrame
    for _, row in df.iterrows():
        # Create a string from each row with no spaces between the values
        row_str = ','.join(map(str, row.values))
        print(row_str)

### CSV

In [ ]:
results = []

for name, df in [('df_ml_wd', df_ml_wd), ('df_ml_wo', df_ml_wo)]:
    is_with_demographics = False
    if 'wd' in name:
        is_with_demographics = True
        
    result = df.groupby('run').apply(calculate_metrics_with_base).reset_index()   
    
    for col in result.columns:
        if 'total_false' in col:
            result[col] = result[col].astype(int)
    
    result['with_demographics'] = is_with_demographics
    results.append(result)

print_df_in_csv_format(df_ml_result)